In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, RepeatVector
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(0)


def n(digits=3):
    number = ''
    for i in range(np.random.randint(1, digits + 1)):
        number += np.random.choice(list('0123456789'))
    return int(number)

Using TensorFlow backend.


In [2]:
a, b =n(), n()
question = '{}+{}'.format(a,b)

In [3]:
question

'5+3'

In [4]:
def padding(chars, maxlen):
    return chars + ' ' * (maxlen - len(chars))

In [7]:
digits = 3
input_digits = digits * 2 + 1
question = padding(question, input_digits)

### 質問と答えのペア

In [9]:
np.random.seed(0)


def n(digits=3):
    number = ''
    for i in range(np.random.randint(1, digits + 1)):
        number += np.random.choice(list('0123456789'))
    return int(number)


def padding(chars, maxlen):
    return chars + ' ' * (maxlen - len(chars))


'''
データの生成
'''
N = 20000
N_train = int(N * 0.9)
N_validation = N - N_train

digits = 3  # 最大の桁数
input_digits = digits * 2 + 1  # 例： 123+456
output_digits = digits + 1  # 500+500 = 1000 以上で４桁になる

added = set()
questions = []
answers = []

while len(questions) < N:
    a, b = n(), n()  # 適当な数を２つ生成

    pair = tuple(sorted((a, b)))
    if pair in added:
        continue

    question = '{}+{}'.format(a, b)
    question = padding(question, input_digits)
    answer = str(a + b)
    answer = padding(answer, output_digits)

    added.add(pair)
    questions.append(question)
    answers.append(answer)

chars = '0123456789+ '
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

X = np.zeros((len(questions), input_digits, len(chars)), dtype=np.integer)
Y = np.zeros((len(questions), digits + 1, len(chars)), dtype=np.integer)

for i in range(N):
    for t, char in enumerate(questions[i]):
        X[i, t, char_indices[char]] = 1
    for t, char in enumerate(answers[i]):
        Y[i, t, char_indices[char]] = 1

X_train, X_validation, Y_train, Y_validation = \
    train_test_split(X, Y, train_size=N_train)

In [14]:
X.shape

(20000, 7, 12)

In [15]:
'''
モデル設定
'''
n_in = len(chars)
n_hidden = 128
n_out = len(chars)

model = Sequential()

# Encoder
model.add(LSTM(n_hidden, input_shape=(input_digits, n_in)))

# Decoder
model.add(RepeatVector(output_digits))
model.add(LSTM(n_hidden, return_sequences=True))

model.add(TimeDistributed(Dense(n_out)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              metrics=['accuracy'])

'''
モデル学習
'''
epochs = 200
batch_size = 200

for epoch in range(epochs):
    model.fit(X_train, Y_train, batch_size=batch_size, epochs=1,
              validation_data=(X_validation, Y_validation))

    # 検証データからランダムに問題を選んで答え合わせ
    for i in range(10):
        index = np.random.randint(0, N_validation)
        question = X_validation[np.array([index])]
        answer = Y_validation[np.array([index])]
        prediction = model.predict_classes(question, verbose=0)

        question = question.argmax(axis=-1)
        answer = answer.argmax(axis=-1)

        q = ''.join(indices_char[i] for i in question[0])
        a = ''.join(indices_char[i] for i in answer[0])
        p = ''.join(indices_char[i] for i in prediction[0])

        print('-' * 10)
        print('Q:  ', q)
        print('A:  ', p)
        print('T/F:', end=' ')
        if a == p:
            print('T')
        else:
            print('F')
    print('-' * 10)

Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 197us/step - loss: 2.0337 - acc: 0.2912 - val_loss: 1.8452 - val_acc: 0.3490
----------
Q:   582+36 
A:   113 
T/F: F
----------
Q:   989+721
A:   1100
T/F: F
----------
Q:   6+575  
A:   11  
T/F: F
----------
Q:   71+68  
A:   11  
T/F: F
----------
Q:   23+8   
A:   11  
T/F: F
----------
Q:   890+79 
A:   110 
T/F: F
----------
Q:   633+23 
A:   11  
T/F: F
----------
Q:   49+77  
A:   11  
T/F: F
----------
Q:   854+89 
A:   113 
T/F: F
----------
Q:   284+712
A:   110 
T/F: F
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 163us/step - loss: 1.7983 - acc: 0.3515 - val_loss: 1.7932 - val_acc: 0.3489
----------
Q:   196+8  
A:   112 
T/F: F
----------
Q:   472+393
A:   110 
T/F: F
----------
Q:   71+74  
A:   11  
T/F: F
----------
Q:   446+8  
A:   13  
T/F: F
----------
Q:   293+3  
A:   13  
T/F: F
--

18000/18000 [==============================] - 3s 187us/step - loss: 1.3215 - acc: 0.5173 - val_loss: 1.3217 - val_acc: 0.5152
----------
Q:   49+32  
A:   30  
T/F: F
----------
Q:   942+74 
A:   902 
T/F: F
----------
Q:   72+809 
A:   899 
T/F: F
----------
Q:   47+81  
A:   115 
T/F: F
----------
Q:   484+56 
A:   582 
T/F: F
----------
Q:   45+70  
A:   112 
T/F: F
----------
Q:   379+7  
A:   300 
T/F: F
----------
Q:   971+29 
A:   900 
T/F: F
----------
Q:   370+89 
A:   311 
T/F: F
----------
Q:   43+0   
A:   44  
T/F: F
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 188us/step - loss: 1.2712 - acc: 0.5363 - val_loss: 1.2651 - val_acc: 0.5328
----------
Q:   37+34  
A:   37  
T/F: F
----------
Q:   0+508  
A:   500 
T/F: F
----------
Q:   138+660
A:   608 
T/F: F
----------
Q:   939+95 
A:   1008
T/F: F
----------
Q:   993+144
A:   1029
T/F: F
----------
Q:   67+380 
A:   411 
T/F: F
----------
Q:   111+

18000/18000 [==============================] - 3s 191us/step - loss: 0.7555 - acc: 0.7234 - val_loss: 0.7871 - val_acc: 0.6944
----------
Q:   6+81   
A:   87  
T/F: T
----------
Q:   338+759
A:   1114
T/F: F
----------
Q:   541+36 
A:   597 
T/F: F
----------
Q:   693+50 
A:   714 
T/F: F
----------
Q:   1+368  
A:   379 
T/F: F
----------
Q:   295+354
A:   741 
T/F: F
----------
Q:   0+590  
A:   590 
T/F: T
----------
Q:   515+371
A:   807 
T/F: F
----------
Q:   48+90  
A:   138 
T/F: T
----------
Q:   832+7  
A:   839 
T/F: T
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 191us/step - loss: 0.7164 - acc: 0.7369 - val_loss: 0.7406 - val_acc: 0.7176
----------
Q:   136+60 
A:   277 
T/F: F
----------
Q:   118+69 
A:   176 
T/F: F
----------
Q:   20+47  
A:   77  
T/F: F
----------
Q:   783+18 
A:   800 
T/F: F
----------
Q:   99+315 
A:   433 
T/F: F
----------
Q:   23+568 
A:   591 
T/F: T
----------
Q:   60+5

18000/18000 [==============================] - 3s 188us/step - loss: 0.3262 - acc: 0.9033 - val_loss: 0.3863 - val_acc: 0.8688
----------
Q:   807+630
A:   1437
T/F: T
----------
Q:   720+3  
A:   723 
T/F: T
----------
Q:   24+654 
A:   688 
T/F: F
----------
Q:   5+182  
A:   187 
T/F: T
----------
Q:   608+92 
A:   791 
T/F: F
----------
Q:   13+843 
A:   856 
T/F: T
----------
Q:   264+253
A:   607 
T/F: F
----------
Q:   824+72 
A:   897 
T/F: F
----------
Q:   478+29 
A:   516 
T/F: F
----------
Q:   69+80  
A:   149 
T/F: T
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 187us/step - loss: 0.3059 - acc: 0.9127 - val_loss: 0.3973 - val_acc: 0.8621
----------
Q:   270+85 
A:   355 
T/F: T
----------
Q:   780+639
A:   1379
T/F: F
----------
Q:   1+505  
A:   506 
T/F: T
----------
Q:   26+31  
A:   57  
T/F: T
----------
Q:   47+791 
A:   858 
T/F: F
----------
Q:   167+102
A:   289 
T/F: F
----------
Q:   39+9

18000/18000 [==============================] - 3s 186us/step - loss: 0.1560 - acc: 0.9645 - val_loss: 0.2490 - val_acc: 0.9165
----------
Q:   193+835
A:   1129
T/F: F
----------
Q:   5+222  
A:   227 
T/F: T
----------
Q:   45+70  
A:   115 
T/F: T
----------
Q:   0+345  
A:   345 
T/F: T
----------
Q:   168+3  
A:   171 
T/F: T
----------
Q:   766+71 
A:   837 
T/F: T
----------
Q:   577+89 
A:   666 
T/F: T
----------
Q:   212+552
A:   754 
T/F: F
----------
Q:   678+2  
A:   680 
T/F: T
----------
Q:   4+990  
A:   995 
T/F: F
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 186us/step - loss: 0.1487 - acc: 0.9654 - val_loss: 0.2499 - val_acc: 0.9142
----------
Q:   660+967
A:   1617
T/F: F
----------
Q:   23+664 
A:   687 
T/F: T
----------
Q:   8+283  
A:   291 
T/F: T
----------
Q:   3+605  
A:   608 
T/F: T
----------
Q:   741+24 
A:   765 
T/F: T
----------
Q:   68+52  
A:   120 
T/F: T
----------
Q:   172+

...

18000/18000 [==============================] - 3s 192us/step - loss: 0.0039 - acc: 1.0000 - val_loss: 0.1596 - val_acc: 0.9533
----------
Q:   45+11  
A:   56  
T/F: T
----------
Q:   83+664 
A:   747 
T/F: T
----------
Q:   535+885
A:   1420
T/F: T
----------
Q:   46+928 
A:   974 
T/F: T
----------
Q:   361+8  
A:   369 
T/F: T
----------
Q:   797+39 
A:   836 
T/F: T
----------
Q:   50+873 
A:   923 
T/F: T
----------
Q:   36+608 
A:   644 
T/F: T
----------
Q:   856+3  
A:   869 
T/F: F
----------
Q:   34+13  
A:   47  
T/F: T
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 198us/step - loss: 0.0037 - acc: 1.0000 - val_loss: 0.1596 - val_acc: 0.9534
----------
Q:   54+221 
A:   275 
T/F: T
----------
Q:   52+354 
A:   406 
T/F: T
----------
Q:   731+261
A:   902 
T/F: F
----------
Q:   146+138
A:   274 
T/F: F
----------
Q:   4+159  
A:   163 
T/F: T
----------
Q:   459+36 
A:   495 
T/F: T
----------
Q:   389+

18000/18000 [==============================] - 3s 187us/step - loss: 0.0047 - acc: 1.0000 - val_loss: 0.1457 - val_acc: 0.9567
----------
Q:   553+9  
A:   562 
T/F: T
----------
Q:   235+963
A:   1208
T/F: F
----------
Q:   33+250 
A:   283 
T/F: T
----------
Q:   641+668
A:   1309
T/F: T
----------
Q:   864+2  
A:   866 
T/F: T
----------
Q:   120+289
A:   319 
T/F: F
----------
Q:   98+19  
A:   117 
T/F: T
----------
Q:   253+317
A:   570 
T/F: T
----------
Q:   7+955  
A:   962 
T/F: T
----------
Q:   893+43 
A:   936 
T/F: T
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 187us/step - loss: 0.0043 - acc: 1.0000 - val_loss: 0.1466 - val_acc: 0.9558
----------
Q:   426+20 
A:   446 
T/F: T
----------
Q:   303+4  
A:   307 
T/F: T
----------
Q:   865+4  
A:   869 
T/F: T
----------
Q:   867+0  
A:   867 
T/F: T
----------
Q:   932+4  
A:   936 
T/F: T
----------
Q:   89+63  
A:   152 
T/F: T
----------
Q:   714+

18000/18000 [==============================] - 3s 193us/step - loss: 0.0024 - acc: 1.0000 - val_loss: 0.1552 - val_acc: 0.9564
----------
Q:   26+444 
A:   470 
T/F: T
----------
Q:   18+815 
A:   843 
T/F: F
----------
Q:   871+1  
A:   872 
T/F: T
----------
Q:   178+7  
A:   185 
T/F: T
----------
Q:   137+37 
A:   174 
T/F: T
----------
Q:   463+445
A:   908 
T/F: T
----------
Q:   801+30 
A:   821 
T/F: F
----------
Q:   858+80 
A:   938 
T/F: T
----------
Q:   559+822
A:   1391
T/F: F
----------
Q:   570+764
A:   1335
T/F: F
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 193us/step - loss: 0.0023 - acc: 1.0000 - val_loss: 0.1557 - val_acc: 0.9550
----------
Q:   704+71 
A:   775 
T/F: T
----------
Q:   4+803  
A:   807 
T/F: T
----------
Q:   5+242  
A:   247 
T/F: T
----------
Q:   627+776
A:   1313
T/F: F
----------
Q:   38+397 
A:   435 
T/F: T
----------
Q:   738+7  
A:   745 
T/F: T
----------
Q:   105+

18000/18000 [==============================] - 3s 190us/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.1637 - val_acc: 0.9558
----------
Q:   30+0   
A:   30  
T/F: T
----------
Q:   97+60  
A:   157 
T/F: T
----------
Q:   382+0  
A:   382 
T/F: T
----------
Q:   160+9  
A:   169 
T/F: T
----------
Q:   35+423 
A:   458 
T/F: T
----------
Q:   728+326
A:   1044
T/F: F
----------
Q:   335+47 
A:   382 
T/F: T
----------
Q:   80+482 
A:   572 
T/F: F
----------
Q:   15+13  
A:   28  
T/F: T
----------
Q:   631+1  
A:   632 
T/F: T
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 192us/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.1607 - val_acc: 0.9561
----------
Q:   56+857 
A:   913 
T/F: T
----------
Q:   968+242
A:   1210
T/F: T
----------
Q:   67+544 
A:   611 
T/F: T
----------
Q:   85+862 
A:   947 
T/F: T
----------
Q:   22+444 
A:   466 
T/F: T
----------
Q:   984+961
A:   1945
T/F: T
----------
Q:   488+

18000/18000 [==============================] - 4s 228us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 0.1689 - val_acc: 0.9560
----------
Q:   902+97 
A:   999 
T/F: T
----------
Q:   9+783  
A:   792 
T/F: T
----------
Q:   3+8    
A:   11  
T/F: T
----------
Q:   0+965  
A:   965 
T/F: T
----------
Q:   36+258 
A:   294 
T/F: T
----------
Q:   85+862 
A:   947 
T/F: T
----------
Q:   35+895 
A:   920 
T/F: F
----------
Q:   216+66 
A:   282 
T/F: T
----------
Q:   693+50 
A:   743 
T/F: T
----------
Q:   608+92 
A:   700 
T/F: T
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 196us/step - loss: 9.8952e-04 - acc: 1.0000 - val_loss: 0.1690 - val_acc: 0.9549
----------
Q:   3+443  
A:   446 
T/F: T
----------
Q:   324+2  
A:   326 
T/F: T
----------
Q:   954+4  
A:   958 
T/F: T
----------
Q:   402+4  
A:   406 
T/F: T
----------
Q:   173+129
A:   202 
T/F: F
----------
Q:   83+36  
A:   119 
T/F: T
----------
Q:   